In [1]:
!pip install transformers==4.37.0
!pip install ktrain==0.26.3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 558.5 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 1.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 596.8 kB/s eta 0:00:0000:0100:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.0
    Uninstalling transformers-4.40.0:
      Successfully uninstalled transformers-4.40.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 2.1 MB/s eta 0:00:0000:0100:01m
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 1.4 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... error
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit

In [2]:
import ktrain
from ktrain import text as ktext


/Users/abd/.pyenv/versions/3.8.10/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import ktrain
from ktrain import text as ktext

# Load your data
df = pd.read_csv("all-data.csv", encoding='latin-1', header=None)
df.columns=["Sentiment", "News Headline"]

label_encoder=LabelEncoder()
df['Sentiment']=label_encoder.fit_transform(df['Sentiment'])

x_train, x_val,y_train,y_val=train_test_split(df['News Headline'], df['Sentiment'], test_size=0.1, random_state=42)

x_train=x_train.tolist()
y_train=y_train.tolist()
x_val=x_val.tolist()
y_val=y_val.tolist()

MODEL_NAME='roberta-base'
trans=ktext.Transformer(MODEL_NAME, maxlen=512, classes=label_encoder.classes_.tolist())

train_data=trans.preprocess_train(x_train, y_train)
val_data =trans.preprocess_test(x_val, y_val)

model =trans.get_classifier()
learner = ktrain.get_learner(model, train_data=train_data, val_data=val_data, batch_size=6)
learner.fit_onecycle(lr=2e-5, epochs=3)

predictor=ktrain.get_predictor(learner.model, preproc=trans)


/Users/abd/.pyenv/versions/3.8.10/lib/python3.8/site-packages/ktrain/text/preprocessor.py:382: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn(
config.json: 100%|██████████████████████████████| 481/481 [00:00<00:00, 371kB/s]
model.safetensors: 100%|█████████████████████| 499M/499M [05:08<00:00, 1.62MB/s]


preprocessing train...
language: en
train sequence lengths:
	mean : 24
	95percentile : 43
	99percentile : 48


tokenizer_config.json: 100%|█████████████████| 25.0/25.0 [00:00<00:00, 13.2kB/s]
vocab.json: 100%|████████████████████████████| 899k/899k [00:00<00:00, 4.35MB/s]
merges.txt: 100%|█████████████████████████████| 456k/456k [00:00<00:00, 716kB/s]
tokenizer.json: 100%|██████████████████████| 1.36M/1.36M [00:01<00:00, 1.33MB/s]


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 24
	95percentile : 43
	99percentile : 51




begin training using onecycle policy with max lr of 2e-05...
Epoch 1/3
61/61 [==============================] - 117s 2s/step - loss: 1.0554 - accuracy: 0.4286 - val_loss: 0.9633 - val_accuracy: 0.5122
Epoch 2/3
61/61 [==============================] - 72s 1s/step - loss: 0.7575 - accuracy: 0.6621 - val_loss: 0.4155 - val_accuracy: 0.8049
Epoch 3/3
61/61 [==============================] - 82s 1s/step - loss: 0.3416 - accuracy: 0.8819 - val_loss: 0.4024 - val_accuracy: 0.8293
